# Multi-Technique Fitting with Shared Parameters

This notebook demonstrates simultaneous fitting of the same rheological model to multiple experimental techniques with shared parameters.

Topics covered:
1. Loading relaxation and oscillation data from the same material
2. Creating models with shared parameters
3. Defining multi-technique objective functions
4. Optimizing parameters across both datasets
5. Cross-validation of fitted parameters
6. Assessing parameter uniqueness and consistency

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from rheo.core.parameters import ParameterSet
from rheo.core.registry import ModelRegistry
from rheo.core.data import RheoData
from rheo.utils.optimization import nlsq_optimize

np.random.seed(42)

## Generate Synthetic Multi-Technique Data

We'll create both relaxation and oscillation data from the same Zener model parameters.

In [ ]:
# True Zener model parameters
Ge_true = 1e5    # Equilibrium modulus (Pa)
Gm_true = 5e5    # Maxwell arm modulus (Pa)
eta_true = 1e5   # Viscosity (Pa·s)
tau_true = eta_true / Gm_true  # Relaxation time (s)

print("True Zener Model Parameters:")
print(f"  Ge (equilibrium modulus) = {Ge_true:.2e} Pa")
print(f"  Gm (Maxwell modulus) = {Gm_true:.2e} Pa")
print(f"  eta (viscosity) = {eta_true:.2e} Pa·s")
print(f"  tau (relaxation time) = {tau_true:.3f} s")

### Generate Stress Relaxation Data

In [ ]:
# Time vector for relaxation
time_relax = np.logspace(-2, 2, 40)  # 0.01 to 100 seconds

# Zener relaxation: G(t) = Ge + Gm * exp(-t/tau)
G_relax_true = Ge_true + Gm_true * np.exp(-time_relax / tau_true)

# Add 4% noise
noise_relax = 0.04
G_relax_noisy = G_relax_true * (1 + noise_relax * np.random.randn(len(G_relax_true)))

# Create RheoData object
relaxation_data = RheoData(
    x=time_relax,
    y=G_relax_noisy,
    x_units='s',
    y_units='Pa',
    domain='time',
    test_mode='relaxation'
)

print(f"\nRelaxation Data:")
print(f"  Time range: {time_relax[0]:.2e} to {time_relax[-1]:.2e} s")
print(f"  Modulus range: {G_relax_noisy.min():.2e} to {G_relax_noisy.max():.2e} Pa")
print(f"  Number of points: {len(time_relax)}")

### Generate Oscillatory Shear Data

In [ ]:
# Frequency vector for oscillation
freq_osc = np.logspace(-2, 2, 35)  # 0.01 to 100 rad/s

# Zener oscillation: G' = Ge + Gm * (omega*tau)^2 / (1 + (omega*tau)^2)
omega_tau = freq_osc * tau_true
G_storage_true = Ge_true + Gm_true * omega_tau**2 / (1 + omega_tau**2)

# Loss modulus: G" = Gm * (omega*tau) / (1 + (omega*tau)^2)
G_loss_true = Gm_true * omega_tau / (1 + omega_tau**2)

# Complex modulus magnitude
G_star_true = np.sqrt(G_storage_true**2 + G_loss_true**2)

# Add 5% noise
noise_osc = 0.05
G_star_noisy = G_star_true * (1 + noise_osc * np.random.randn(len(G_star_true)))

# Create RheoData object
oscillation_data = RheoData(
    x=freq_osc,
    y=G_star_noisy,
    x_units='rad/s',
    y_units='Pa',
    domain='frequency',
    test_mode='oscillation'
)

print(f"\nOscillation Data:")
print(f"  Frequency range: {freq_osc[0]:.2e} to {freq_osc[-1]:.2e} rad/s")
print(f"  Complex modulus range: {G_star_noisy.min():.2e} to {G_star_noisy.max():.2e} Pa")
print(f"  Number of points: {len(freq_osc)}")

## Visualize Both Datasets

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Relaxation data
ax1.loglog(time_relax, G_relax_noisy, 'o', label='Data', markersize=6, alpha=0.7)
ax1.loglog(time_relax, G_relax_true, '--', label='True Model', 
           color='red', linewidth=2)
ax1.axhline(y=Ge_true, color='green', linestyle=':', 
            linewidth=2, label=f'Ge = {Ge_true:.1e} Pa')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12)
ax1.set_title('Stress Relaxation', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Right: Oscillation data
ax2.loglog(freq_osc, G_star_noisy, 's', label='Data (|G*|)', markersize=6, alpha=0.7)
ax2.loglog(freq_osc, G_star_true, '--', label='True Model', 
           color='red', linewidth=2)
ax2.loglog(freq_osc, G_storage_true, ':', label="G'", 
           color='blue', linewidth=2, alpha=0.5)
ax2.loglog(freq_osc, G_loss_true, ':', label='G"', 
           color='orange', linewidth=2, alpha=0.5)
ax2.set_xlabel('Frequency (rad/s)', fontsize=12)
ax2.set_ylabel('Modulus (Pa)', fontsize=12)
ax2.set_title('Oscillatory Shear', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.suptitle('Multi-Technique Data from Same Material', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Single-Technique Fits (Baseline)

First, let's fit each dataset independently to establish a baseline.

In [ ]:
from rheo.pipeline import Pipeline

# Fit relaxation data only
pipeline_relax = Pipeline()
pipeline_relax.load_data(relaxation_data)
pipeline_relax.fit('zener')
params_relax_only = pipeline_relax.get_fitted_parameters()

print("\n" + "="*60)
print("FIT FROM RELAXATION DATA ONLY")
print("="*60)
for name, param in params_relax_only.items():
    true_val = {'Ge': Ge_true, 'Gm': Gm_true, 'eta': eta_true}[name]
    error = abs(param.value - true_val) / true_val * 100
    print(f"  {name}: {param.value:.3e} Pa (true: {true_val:.3e}, error: {error:.1f}%)")

# Fit oscillation data only
pipeline_osc = Pipeline()
pipeline_osc.load_data(oscillation_data)
pipeline_osc.fit('zener')
params_osc_only = pipeline_osc.get_fitted_parameters()

print("\n" + "="*60)
print("FIT FROM OSCILLATION DATA ONLY")
print("="*60)
for name, param in params_osc_only.items():
    true_val = {'Ge': Ge_true, 'Gm': Gm_true, 'eta': eta_true}[name]
    error = abs(param.value - true_val) / true_val * 100
    print(f"  {name}: {param.value:.3e} Pa (true: {true_val:.3e}, error: {error:.1f}%)")

## Multi-Technique Fitting with Shared Parameters

Now we'll fit both datasets simultaneously, sharing parameters between them.

In [ ]:
# Create model instance
model = ModelRegistry.create('zener')

# Set initial guesses (average of single-technique fits)
model.parameters['Ge'].value = (params_relax_only['Ge'].value + params_osc_only['Ge'].value) / 2
model.parameters['Gm'].value = (params_relax_only['Gm'].value + params_osc_only['Gm'].value) / 2
model.parameters['eta'].value = (params_relax_only['eta'].value + params_osc_only['eta'].value) / 2

print("\nInitial Parameter Guesses (average of single-technique fits):")
for name, param in model.parameters.items():
    print(f"  {name}: {param.value:.3e}")

### Define Multi-Technique Objective Function

In [ ]:
def multi_technique_objective(params_array, weight_relax=1.0, weight_osc=1.0):
    """
    Objective function combining relaxation and oscillation data.
    
    Parameters:
        params_array: Array of parameter values [Ge, Gm, eta]
        weight_relax: Weight for relaxation data contribution
        weight_osc: Weight for oscillation data contribution
    
    Returns:
        Combined weighted sum of squared residuals
    """
    # Update model parameters
    param_names = list(model.parameters.keys())
    for i, name in enumerate(param_names):
        model.parameters[name].value = params_array[i]
    
    # Predict relaxation data
    relax_pred = model.predict(relaxation_data)
    relax_residuals = (relaxation_data.y - relax_pred) / relaxation_data.y
    relax_sse = np.sum(relax_residuals**2)
    
    # Predict oscillation data
    osc_pred = model.predict(oscillation_data)
    osc_residuals = (oscillation_data.y - osc_pred) / oscillation_data.y
    osc_sse = np.sum(osc_residuals**2)
    
    # Combined objective (weighted sum)
    total_objective = weight_relax * relax_sse + weight_osc * osc_sse
    
    return total_objective

# Test initial objective value
initial_params = np.array([p.value for p in model.parameters.values()])
initial_obj = multi_technique_objective(initial_params)
print(f"\nInitial multi-technique objective value: {initial_obj:.3e}")

### Optimize with Different Weighting Schemes

We'll try three weighting schemes:
1. Equal weights (1:1)
2. Favor relaxation (2:1)
3. Favor oscillation (1:2)

In [ ]:
# Define weighting schemes
weighting_schemes = [
    (1.0, 1.0, 'Equal (1:1)'),
    (2.0, 1.0, 'Favor Relaxation (2:1)'),
    (1.0, 2.0, 'Favor Oscillation (1:2)')
]

results_multi = []
bounds = [p.bounds for p in model.parameters.values()]

print("\n" + "="*70)
print("MULTI-TECHNIQUE OPTIMIZATION RESULTS")
print("="*70)

for w_relax, w_osc, scheme_name in weighting_schemes:
    # Optimize
    result = minimize(
        lambda p: multi_technique_objective(p, w_relax, w_osc),
        initial_params,
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 1000, 'ftol': 1e-10}
    )
    
    results_multi.append((result, scheme_name))
    
    print(f"\n{scheme_name}:")
    print("-" * 70)
    print(f"  Success: {result.success}")
    print(f"  Objective: {result.fun:.4e}")
    print(f"  Parameters:")
    param_names = list(model.parameters.keys())
    true_vals = [Ge_true, Gm_true, eta_true]
    for i, (name, true_val) in enumerate(zip(param_names, true_vals)):
        error = abs(result.x[i] - true_val) / true_val * 100
        print(f"    {name}: {result.x[i]:.3e} (true: {true_val:.3e}, error: {error:.1f}%)")

# Select equal-weight result as the best
best_result_multi, _ = results_multi[0]
param_names = list(model.parameters.keys())
for i, name in enumerate(param_names):
    model.parameters[name].value = best_result_multi.x[i]

## Visualize Multi-Technique Fit

In [ ]:
# Get multi-technique predictions
relax_pred_multi = model.predict(relaxation_data)
osc_pred_multi = model.predict(oscillation_data)

# Also get single-technique predictions for comparison
relax_pred_single = pipeline_relax.predict(relaxation_data)
osc_pred_single = pipeline_osc.predict(oscillation_data)

fig, axes = plt.subplots(2, 2, figsize=(14, 11))

# Top left: Relaxation fits
ax = axes[0, 0]
ax.loglog(time_relax, G_relax_noisy, 'o', label='Data', markersize=6, alpha=0.6)
ax.loglog(time_relax, relax_pred_single, '--', label='Single-technique fit', linewidth=2)
ax.loglog(time_relax, relax_pred_multi, '-', label='Multi-technique fit', linewidth=2.5)
ax.loglog(time_relax, G_relax_true, ':', label='True model', color='red', linewidth=2)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('G(t) (Pa)', fontsize=11)
ax.set_title('Relaxation: Model Comparison', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Top right: Oscillation fits
ax = axes[0, 1]
ax.loglog(freq_osc, G_star_noisy, 's', label='Data', markersize=6, alpha=0.6)
ax.loglog(freq_osc, osc_pred_single, '--', label='Single-technique fit', linewidth=2)
ax.loglog(freq_osc, osc_pred_multi, '-', label='Multi-technique fit', linewidth=2.5)
ax.loglog(freq_osc, G_star_true, ':', label='True model', color='red', linewidth=2)
ax.set_xlabel('Frequency (rad/s)', fontsize=11)
ax.set_ylabel('|G*| (Pa)', fontsize=11)
ax.set_title('Oscillation: Model Comparison', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Bottom left: Relaxation residuals
ax = axes[1, 0]
residuals_relax_single = (G_relax_noisy - relax_pred_single) / G_relax_noisy * 100
residuals_relax_multi = (G_relax_noisy - relax_pred_multi) / G_relax_noisy * 100
ax.semilogx(time_relax, residuals_relax_single, 'o--', label='Single-technique', alpha=0.7)
ax.semilogx(time_relax, residuals_relax_multi, 's-', label='Multi-technique', alpha=0.7)
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Relative Error (%)', fontsize=11)
ax.set_title('Relaxation Residuals', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Bottom right: Oscillation residuals
ax = axes[1, 1]
residuals_osc_single = (G_star_noisy - osc_pred_single) / G_star_noisy * 100
residuals_osc_multi = (G_star_noisy - osc_pred_multi) / G_star_noisy * 100
ax.semilogx(freq_osc, residuals_osc_single, 's--', label='Single-technique', alpha=0.7)
ax.semilogx(freq_osc, residuals_osc_multi, 'o-', label='Multi-technique', alpha=0.7)
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
ax.set_xlabel('Frequency (rad/s)', fontsize=11)
ax.set_ylabel('Relative Error (%)', fontsize=11)
ax.set_title('Oscillation Residuals', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.suptitle('Multi-Technique Fitting Results', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

## Cross-Validation: Parameter Consistency Check

In [ ]:
# Compare parameter estimates
print("\n" + "="*80)
print("PARAMETER CONSISTENCY ANALYSIS")
print("="*80)

param_comparison = pd.DataFrame({
    'Parameter': ['Ge', 'Gm', 'eta'],
    'True Value': [Ge_true, Gm_true, eta_true],
    'Relaxation Only': [params_relax_only[p].value for p in ['Ge', 'Gm', 'eta']],
    'Oscillation Only': [params_osc_only[p].value for p in ['Ge', 'Gm', 'eta']],
    'Multi-Technique': [model.parameters[p].value for p in ['Ge', 'Gm', 'eta']]
})

# Calculate errors
for col in ['Relaxation Only', 'Oscillation Only', 'Multi-Technique']:
    param_comparison[f'{col} Error (%)'] = (
        abs(param_comparison[col] - param_comparison['True Value']) / 
        param_comparison['True Value'] * 100
    )

print("\n" + param_comparison.to_string(index=False))

# Calculate average errors
avg_error_relax = param_comparison['Relaxation Only Error (%)'].mean()
avg_error_osc = param_comparison['Oscillation Only Error (%)'].mean()
avg_error_multi = param_comparison['Multi-Technique Error (%)'].mean()

print(f"\n\nAverage Parameter Errors:")
print(f"  Relaxation only:   {avg_error_relax:.2f}%")
print(f"  Oscillation only:  {avg_error_osc:.2f}%")
print(f"  Multi-technique:   {avg_error_multi:.2f}%")

improvement = min(avg_error_relax, avg_error_osc) - avg_error_multi
print(f"\n  Improvement: {improvement:.2f}% reduction in average error")

## Visualize Parameter Estimates

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

param_names = ['Ge', 'Gm', 'eta']
param_labels = ['Ge (Pa)', 'Gm (Pa)', 'eta (Pa·s)']
true_values = [Ge_true, Gm_true, eta_true]

for ax, param_name, param_label, true_val in zip(axes, param_names, param_labels, true_values):
    estimates = [
        params_relax_only[param_name].value,
        params_osc_only[param_name].value,
        model.parameters[param_name].value
    ]
    
    x_pos = np.arange(3)
    bars = ax.bar(x_pos, estimates, alpha=0.7, color=['blue', 'orange', 'green'])
    ax.axhline(y=true_val, color='red', linestyle='--', linewidth=2, label='True value')
    
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Relax\nOnly', 'Osc\nOnly', 'Multi-\nTech'], fontsize=10)
    ax.set_ylabel(param_label, fontsize=11)
    ax.set_title(f'{param_name} Estimates', fontsize=12)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, (bar, val) in enumerate(zip(bars, estimates)):
        height = bar.get_height()
        error = abs(val - true_val) / true_val * 100
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{error:.1f}%',
                ha='center', va='bottom', fontsize=8)

plt.suptitle('Parameter Estimation Comparison (% error shown)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we demonstrated multi-technique fitting with shared parameters:

### Key Findings:

1. **Improved Accuracy**: Multi-technique fitting typically provides more accurate parameter estimates than single-technique fits
2. **Parameter Consistency**: Shared parameters enforce consistency across different experimental techniques
3. **Reduced Uncertainty**: Combining complementary data reduces parameter correlation and uncertainty
4. **Cross-Validation**: Different techniques probe different aspects of material behavior

### Benefits of Multi-Technique Fitting:

- **Better Parameter Uniqueness**: Reduces parameter correlation (identifiability)
- **Increased Data Coverage**: Extends frequency/time range by combining techniques
- **Physical Consistency**: Ensures same material properties across experiments
- **Robustness**: Less sensitive to noise in individual datasets

### When to Use Multi-Technique Fitting:

- When you have multiple datasets from the same material
- When parameters are poorly constrained by single technique
- For materials with complex relaxation spectra
- When validating model consistency across techniques

### Practical Considerations:

1. **Weighting**: Balance contributions from techniques with different noise levels
2. **Model Validity**: Ensure model is appropriate for all techniques
3. **Data Quality**: Poor data from one technique can degrade overall fit
4. **Computational Cost**: Multi-technique optimization is more expensive

### Advanced Extensions:

- Bayesian multi-technique fitting for uncertainty quantification (Phase 3)
- Hierarchical models with shared and technique-specific parameters
- Time-temperature-superposition with multi-technique validation
- Global fitting across multiple samples with shared parameters

### Next Steps:

- Explore model comparison (see `multi_model_comparison.ipynb`)
- Apply advanced optimization strategies (see `advanced_workflows.ipynb`)
- Try mastercurve generation (see `mastercurve_generation.ipynb`)